#Cargue de los dataframes de proccessed en una base de datos SQL Lite

#1.Instalación de paquetes e importación de librerías

In [1]:
#Se importan las librerías a utilizar 
import requests
import pprint
import pandas as pd
import json
import glob
import sys
from datetime import date, time, datetime
import sqlite3

#2.Se monta el drive 

In [2]:
#Se monta la unidad de drive con el fin de extraer los datos que almacenamos en nuestro Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#3.Lectura de archivos csv

In [3]:
#Se lee el csv de la tabla de Género
Genero = pd.read_csv("/content/drive/MyDrive/Prueba_Tecnica_Lulo_Blank/Consumo de datos/Simulacion_Data_Lake/2.Proccesed/Genero/Genero.csv", sep=',')
Genero=Genero[['Id_Genero','Genero']]

#Se lee le archivo para la dimensión de de Dim_Show
dim_show = pd.read_csv("/content/drive/MyDrive/Prueba_Tecnica_Lulo_Blank/Consumo de datos/Simulacion_Data_Lake/2.Proccesed/Embedded_show/embedded_show.csv", sep=',')

dim_show = dim_show.drop(['embedded_show_network'], axis=1)
dim_show = dim_show.astype({'embedded_show_weight':'float64'})

#Se lee le archivo para la dimensión de de Dim_Net_Work
Dim_Net_Work = pd.read_csv("/content/drive/MyDrive/Prueba_Tecnica_Lulo_Blank/Consumo de datos/Simulacion_Data_Lake/2.Proccesed/Embedded_show_net_work/embedded_show_net_work.csv", sep=',')

#Se lee el archivo para la dimensión de 
WebChannel= pd.read_csv("/content/drive/MyDrive/Prueba_Tecnica_Lulo_Blank/Consumo de datos/Simulacion_Data_Lake/2.Proccesed/Embedded_show_Web_Channel/embedded_show_web_channel.csv", sep=',')
WebChannel =WebChannel.astype({'embedded_show_webChannel_id':'int64'})

#Se lee el archivo para la construcción de la tabla de Fact Series 
Fact_Series = pd.read_csv("/content/drive/MyDrive/Prueba_Tecnica_Lulo_Blank/Consumo de datos/Simulacion_Data_Lake/2.Proccesed/Series/Series.csv", sep=',')

Fact_Series = Fact_Series.astype({'airdate':'datetime64',
                                        'airtime':'datetime64', 
                                        'airstamp':'datetime64',
                                        'image':'str',
                                        'embedded_show_dvdCountry':'str', 
                                        'embedded_show_webChannel':'str',
                                        'embedded_show_image':'str',
                                        'embedded_show_webChannel_country':'str'
                                       })

# Se lee el archivo para la construcción de la tabla de Dim_Tiempo
Dim_Tiempo = pd.read_csv("/content/drive/MyDrive/Prueba_Tecnica_Lulo_Blank/Consumo de datos/Simulacion_Data_Lake/2.Proccesed/Dim_Tiempo/Dim_Tiempo.csv", sep=';')
Dim_Tiempo=Dim_Tiempo.drop(['Week of Month','Day Name','Day of Week'],axis=1)


#4.Escritura en base de datos en SQL lite de Dim_Genero

In [4]:
#Crear base de datos 
conn = sqlite3.connect('BD_Series.db')

In [5]:

##Se crea la tabla de genero 
create_sql="CREATE TABLE IF NOT EXISTS dim_genero (Id_Genero INTEGER NOT NULL PRIMARY KEY, Genero TEXT)"
cursor=conn.cursor()
cursor.execute(create_sql)

#Se llena la tabla de genero con el dataframe 
for row in Genero.itertuples():
  insert_sql=f"INSERT INTO dim_genero (Id_Genero, Genero) VALUES ({row[1]},'{row[2]}')"
  cursor.execute(insert_sql)

#Se hace commit o actualiza la base de datos 
conn.commit()

#5.Escritura en base de datos en SQL lite de Dim_Tiempo

In [6]:
##Se crea la tabla de genero 
create_sql="CREATE TABLE IF NOT EXISTS dim_tiempo (Date TEXT NOT NULL PRIMARY KEY, Day INTEGER, DayName TEXT, WeekDayNum INTEGER, Quarter INTEGER, Year INTEGER, Year_half INTEGER, MonthNum INTEGER, MonthLong TEXT, Month TEXT, MonthYear TEXT, MonthYearLong TEXT )"
cursor=conn.cursor()
cursor.execute(create_sql)

#Se llena la tabla de genero con el dataframe 
for row in Dim_Tiempo.itertuples():
  insert_sql=f"INSERT INTO dim_tiempo (Date, Day, DayName, WeekDayNum, Quarter, Year, Year_half, MonthNum, MonthLong, Month, MonthYear, MonthYearLong) VALUES ('{row[1]}' , {row[2]}, '{row[3]}' , {row[4]} , {row[5]} , {row[6]} , {row[7]} , {row[8]} , '{row[9]}' , '{row[10]}' , '{row[11]}' , '{row[12]}')"
  cursor.execute(insert_sql)

#Se hace commit o actualiza la base de datos 
conn.commit()

#6.Construcción y escritura de la tabla de pais 

In [7]:
#Seleccion de las columnas de country de Dim_Net_Work
df1=Dim_Net_Work[['embedded_show_network_country_cod','embedded_show_network_country_timezone','embedded_show_network_country_name']]

#Renombrar columnas 
df1=df1.rename(columns={'embedded_show_network_country_cod':'Cod_Country',
                        'embedded_show_network_country_timezone':'timezone',
                        'embedded_show_network_country_name':'Country_name'})

#Selección de las columnas de WebChannel 
df2 = WebChannel[['embedded_show_webChannel_country_code','embedded_show_webChannel_country_timezone','embedded_show_webChannel_country_name']]

#Renombrar columnas 
df2 = df2.rename(columns={'embedded_show_webChannel_country_code':'Cod_Country',
                        'embedded_show_webChannel_country_timezone':'timezone',
                        'embedded_show_webChannel_country_name':'Country_name'})

#Selección de las columnas 
df3 = Fact_Series[['embedded_show_dvdCountry_code','embedded_show_dvdCountry_timezone','embedded_show_dvdCountry_name']]

#Renombrar columnas 
df3 = df3.rename(columns={'embedded_show_dvdCountry_code':'Cod_Country',
                        'embedded_show_dvdCountry_timezone':'timezone',
                        'embedded_show_dvdCountry_name':'Country_name'})

#Hacer append entre dataframes
Dim_Country=pd.concat([df1,df2,df3])

#Eliminar duplicados
Dim_Country = Dim_Country.drop_duplicates(subset=['Cod_Country'])
#Reiniciar el índice
Dim_Country.reset_index(level=None, drop=True, inplace=True, col_level=0, col_fill='')


##Se crea la tabla de Country
create_sql="CREATE TABLE IF NOT EXISTS dim_country ( Cod_Country TEXT NOT NULL PRIMARY KEY, timezone TEXT, Country_name TEXT )"
cursor=conn.cursor()
cursor.execute(create_sql)

#Se llena la tabla de genero con el dataframe 
for row in Dim_Country.itertuples():
  insert_sql=f"INSERT INTO dim_country (Cod_Country, timezone, Country_name) VALUES ('{row[1]}' , '{row[2]}', '{row[3]}')"
  cursor.execute(insert_sql)

#Se hace commit o actualiza la base de datos 
conn.commit()

#7.Escritura en base de datos en SQL lite de Dim_Webchannel 

In [8]:
Dim_WebChannel=WebChannel[['embedded_show_webChannel_id','embedded_show_webChannel_name','embedded_show_webChannel_officialSite','embedded_show_webChannel_country_code']]

##Se crea la tabla de Dim_Webchannel 
create_sql="CREATE TABLE IF NOT EXISTS dim_web_channel ( embedded_show_webChannel_id INTEGER NOT NULL PRIMARY KEY, embedded_show_webChannel_name TEXT, embedded_show_webChannel_officialSite TEXT ,embedded_show_webChannel_country_code TEXT, FOREIGN KEY (embedded_show_webChannel_country_code) REFERENCES dim_country(Cod_Country))"
cursor=conn.cursor()
cursor.execute(create_sql)

#Se llena la tabla de genero con el dataframe 
for row in Dim_WebChannel.itertuples():
  insert_sql=f"INSERT INTO dim_web_channel (embedded_show_webChannel_id, embedded_show_webChannel_name, embedded_show_webChannel_officialSite, embedded_show_webChannel_country_code) VALUES ({row[1]}, '{row[2]}', '{row[3]}','{row[4]}')"
  cursor.execute(insert_sql)

#Se hace commit o actualiza la base de datos 
conn.commit()


#8.Escritura en base de datos en SQL lite de Dim_Net_Work

In [9]:
Dim_Net_Work = Dim_Net_Work[['embedded_show_network_id','embedded_show_network_name', 'embedded_show_network_officialSite','embedded_show_network_country_cod']]

##Se crea la tabla de Dim_Webchannel 
create_sql="CREATE TABLE IF NOT EXISTS dim_net_work ( embedded_show_network_id INTEGER NOT NULL PRIMARY KEY, embedded_show_network_name TEXT, embedded_show_network_officialSite TEXT ,embedded_show_network_country_cod TEXT, FOREIGN KEY (embedded_show_network_country_cod) REFERENCES dim_country(Cod_Country))"
cursor=conn.cursor()
cursor.execute(create_sql)

#Se llena la tabla de genero con el dataframe 
for row in Dim_Net_Work.itertuples():
  insert_sql=f"INSERT INTO dim_net_work (embedded_show_network_id, embedded_show_network_name, embedded_show_network_officialSite, embedded_show_network_country_cod ) VALUES ({row[1]}, '{row[2]}', '{row[3]}','{row[4]}')"
  cursor.execute(insert_sql)

#Se hace commit o actualiza la base de datos 
conn.commit()

#9.Escritura en base de datos en SQL lite de Dim_Show

In [10]:
##Hace falta rellenar la tabla como las otras para poder marcar las llaves primarias y llaves foráneas 
dim_show.to_sql(name="dim_show",con=conn,if_exists='replace',index=False)
conn.commit()

#10.Escritura en base de datos en SQL lite de Fact_Series

In [13]:
##Hace falta rellenar la tabla como las otras para poder marcar las llaves primarias y llaves foráneas 
Fact_Series.to_sql(name="fact_series", con=conn, if_exists='replace', index=False)
conn.commit()